# StackExchange questions and Tags - Modélisation

This Notebook is about Stackoverflow questions tag prediction 
- I start by importing libraries and Data 
- I make in this Notebook a first unsupervised approch with LDA

## Libraries and frameworks

In [1]:
!pip install tqdm
!pip install flake8 --ignore-installed
!pip install pycodestyle_magic
!pip install wordcloud
!pip install gensim
!pip install pyLDAvis

  Using cached https://files.pythonhosted.org/packages/f8/1f/7ea40d1e4146ea55dbab41cda1376db092a75794914169aabd7e8d7a7def/flake8-3.7.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/f2/ed0ffb887f8138a8fe5a621b8c0bb9598bfb3989e029f6c6a85ee66628ee/pyflakes-2.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/0c/04a353e104d2f324f8ee5f4b32012618c1c86dd79e52a433b64fceed511b/pycodestyle-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/89/479dc97e18549e21354893e4ee4ef36db1d237534982482c3681ee6e7b57/mccabe-0.6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/c6/44694103f8c221443ee6b0041f69e2740d89a25641e62fb4f2ee568f2f9c/entrypoints-0.3-py2.py3-none-any.whl


ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [21]:
import pandas as pd
import numpy as np
from pprint import pprint

from tqdm import tqdm

from bs4 import BeautifulSoup
import heapq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import scipy.sparse
import pickle

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn  # don't skip this
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib import pyplot as plt

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s \
: %(levelname)s : %(message)s', level=logging.ERROR)

In [5]:
%load_ext pycodestyle_magic
%pycodestyle_on
%matplotlib inline

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

pyLDAvis.enable_notebook()

## Data

In [6]:
df = pd.read_csv('C:/Users/ayoub/OCP6 Data/df.csv', sep=',')

In [7]:
df.head()

,Body,Title,Tags,AnswerCount,FavoriteCount,tokens,title_tokens,tags,ntags,t,sentence,tsentence
0,I want to use a track-bar to change a form's o...,Convert Decimal to Double,<c#><floating-point><type-conversion><double><...,13,50,"['want', 'use', 'track-bar', 'chang', ""form'"",...","['convert', 'decim', 'doubl']","['c#', 'floating-point', 'type-conversion', 'd...",5,"['convert', 'decim', 'doubl', 'want', 'use', '...",convert decim doubl want use track-bar chang f...,c# floating-point type-conversion double decimal
1,I have an absolutely positioned div containing...,Percentage width child element in absolutely p...,<html><css><internet-explorer-7>,6,11,"['absolut', 'posit', 'div', 'contain', 'sever'...","['percentag', 'width', 'child', 'element', 'ab...","['html', 'css', 'internet-explorer-7']",3,"['percentag', 'width', 'child', 'element', 'ab...",percentag width child element absolut_posit pa...,html css internet-explorer-7
2,Given a DateTime representing a person's birth...,How do I calculate someone's age in C#?,<c#><.net><datetime>,61,443,"['given', 'datetim', 'repres', ""person'"", 'bir...","['calcul', ""someone'"", 'age', 'c#?']","['c#', '.net', 'datetime']",3,"['calcul', ""someone'"", 'age', 'c#?', 'given', ...",calcul someone' age c#? given datetim repres p...,c# .net datetime
3,"Given a specific DateTime value, how do I disp...",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,37,540,"['given', 'specif', 'datetim', 'value,', 'disp...","['calcul', 'rel', 'time', 'c#']","['c#', 'datetime', 'time', 'datediff', 'relati...",5,"['calcul', 'rel', 'time', 'c#', 'given', 'spec...","calcul rel time c# given specif datetim value,...",c# datetime time datediff relative-time-span
4,Is there a standard way for a web server to be...,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,24,147,"['standard', 'way', 'web', 'server', 'abl', 'd...","['determin', ""user'"", 'timezon']","['html', 'browser', 'timezone', 'user-agent', ...",5,"['determin', ""user'"", 'timezon', 'standard', '...",determin user' timezon standard way web server...,html browser timezone user-agent timezone-offset


In [8]:
df.columns

Index(['Body', 'Title', 'Tags', 'AnswerCount', 'FavoriteCount', 'tokens',
       'title_tokens', 'tags', 'ntags', 't', 'sentence', 'tsentence'],
      dtype='object')

In [9]:
X_C = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//X_C.npz')
X_T = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//X_T.npz')
Y_C = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//Y_C.npz')
#Y_T = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//Y_T.npz')

INFO:pycodestyle:4:1: E265 block comment should start with '# '


In [10]:
with open("C:/Users/ayoub/OCP6 Data/X_C_columns.txt", "rb") as fp:
    X_C_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/X_T_columns.txt", "rb") as fp:
    X_T_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/Y_C_columns.txt", "rb") as fp:
    Y_C_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/Y_T_columns.txt", "rb") as fp:
    Y_T_columns = pickle.load(fp)

In [13]:
X_C.shape, X_T.shape, Y_C.shape

((103676, 31541), (103676, 31541), (103676, 2639))

In [14]:
X_C.dtype

dtype('int64')

In [16]:
pd.DataFrame(Y_C.toarray(), columns = Y_C_columns).head()

,.htaccess,.net,.net-1.1,.net-2.0,.net-3.0,.net-3.5,.net-4.0,2d,32-bit,32bit-64bit,3d,64-bit,abap,absolute-path,abstract,abstract-class,abstract-syntax-tree,abstraction,acceptance-testing,access-control,access-modifiers,access-vba,access-violation,accessibility,accessor,accordion,acl,action,actionlink,actionresult,actionscript,actionscript-2,actionscript-3,active-directory,activemq,activerecord,activeresource,activex,adam,add-in,add-on,admin,administration,ado,ado.net,adobe,ads,adsi,aes,aggregate,aggregate-functions,aggregation,agile,air,aix,ajax,ajaxcontroltoolkit,alerts,algebra,algorithm,alias,alignment,allocation,alphablending,alter-table,amazon,amazon-ec2,amazon-s3,amazon-simpledb,amazon-web-services,amf,analysis,analytics,anchor,android,android-activity,angle,animated-gif,animation,ankhsvn,annotations,anonymous-class,anonymous-methods,anonymous-types,ant,anti-patterns,antialiasing,antivirus,antlr,aop,apache,apache-commons,apache-flex,apache-poi,apache2,apc,api,api-design,app-config,app-store,...,windbg,window,windows,windows-7,windows-authentication,windows-ce,windows-explorer,windows-forms-designer,windows-installer,windows-mobile,windows-server-2003,windows-server-2008,windows-services,windows-shell,windows-vista,windows-xp,winforms,winsock,wireless,wix,wix3,wizard,wmd,wmi,wndproc,wolfram-mathematica,word-vba,word-wrap,wordpress,workflow,workflow-foundation,worksheet-function,workspace,wpf,wpf-controls,wpftoolkit,wrapper,ws-security,wsdl,wse,wsgi,wsh,wss,wss-3.0,wxpython,wxwidgets,wysiwyg,x11,x509,x86,x86-64,xaml,xampp,xbap,xbox360,xcode,xdebug,xhtml,xib,xls,xml,xml-attribute,xml-comments,xml-documentation,xml-namespaces,xml-rpc,xml-serialization,xmldocument,xmlhttprequest,xmlreader,xmpp,xna,xor,xpath,xps,xquery,xsd,xsl-fo,xslt,xslt-1.0,xslt-2.0,xss,xstream,xul,xulrunner,yacc,yahoo,yahoo-pipes,yaml,yield,youtube,yui,z-index,zend-db-table,zend-form,zend-framework,zip,zlib,zoom,zsh
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


INFO:pycodestyle:1:36: E251 unexpected spaces around keyword / parameter equals
INFO:pycodestyle:1:38: E251 unexpected spaces around keyword / parameter equals


## Unsupervised approach

### Building the Topic Model with LDA 

In [22]:
Cvectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), max_df=0.5, min_df=10)
X = Cvectorizer.fit_transform(df.sentence)

INFO:pycodestyle:1:80: E501 line too long (86 > 79 characters)


In [23]:
lda_tf = LatentDirichletAllocation(n_components=30, random_state=0)
lda_tf.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=30, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [24]:
data = pyLDAvis.sklearn.prepare(lda_tf, X, Cvectorizer, mds='mmds')

C:\Users\ayoub\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [05/Feb/2020 17:49:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 17:49:33] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 17:49:33] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 17:49:33] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2020 17:49:33] code 404, message Not Found
127.0.0.1 - - [05/Feb/2020 17:49:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Feb/2020 18:16:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2020 18:16:30] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2020 18:16:30] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2020 18:16:30] "GET /LDAvis.js HTTP/1.1" 200 -


In [30]:
X.shape

(103676, 31541)

After playing with pyLDavis, i would say that it will be complicated to do all the work and predict tags with an unsupervised approch, so I will try some classification methods

## Supervised approach

Because of a memory problem we have to reduce the data and do the calculation on a stronger instance (AWS),

Go to Supervised approach Notebook :)

# Brouillon

In [ ]:
Cvectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), max_df=0.5, min_df=10)
X = Cvectorizer.fit_transform(df_test.sentence)

In [ ]:
lda_tf = LatentDirichletAllocation(n_components=30, random_state=0)
lda_tf.fit(X)

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, X, Cvectorizer, mds='mmds')